## Loading libraries

In [4]:
import requests
from requests.auth import HTTPBasicAuth
from lxml import html
from cssselect import GenericTranslator
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import json
import os

In [5]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', None)

In [6]:
if not os.path.exists('data'):
    print(os.path.abspath('.'))

/Users/rubencito/CAS_datascience/ADS_CAS_Bern_2020/Projects/M3


### Collecting the **Data Catalog** from the UNDP (Human Development Report Office Statistical Data API)

In [7]:
my_data_catalog_path = '/Users/rubencito/CAS_datascience/ADS_CAS_Bern_2020/Projects/M3/UNDP_data_catalog/HDRO.html'

In [8]:
# loading the Indicator parameters
indi_col_names = ['ID', 'Indicator_Name']
indi_UNDP_df = pd.read_html(my_data_catalog_path)[0][2:]
indi_UNDP_df.reset_index(inplace = True, drop = True) # fix the indexing
indi_UNDP_df.columns = indi_col_names # Fix columns names
indi_UNDP_df.head(6)

,ID,Indicator_Name
0,164406,Adjusted net savings (% of GNI)
1,36806,"Adolescent birth rate (births per 1,000 women ages 15-19)"
2,185106,"Age-standardized mortality rate attributed to noncommunicable diseases, female"
3,185206,"Age-standardized mortality rate attributed to noncommunicable diseases, male"
4,175206,"Antenatal care coverage, at least one visit (%)"
5,186806,Average annual change in the share of bottom 40 percent (%)


In [12]:
indi_UNDP_df[indi_UNDP_df['Indicator_Name'].str.contains("multi", case = False)]

,ID,Indicator_Name
15,117806,Contribution of deprivation in education to the Multidimensional Poverty Index
16,117906,Contribution of deprivation in health to the Multidimensional Poverty Index
17,118006,Contribution of deprivation in standard of living to the Multidimensional Poverty Index
99,38406,Multidimensional poverty index (MPI)
112,38606,"Population in multidimensional poverty, headcount (%)"
113,102006,"Population in multidimensional poverty, headcount (thousands) (for the year of the survey)"
114,183406,"Population in multidimensional poverty, headcount (thousands) (projection for 2018)"
115,38506,"Population in multidimensional poverty, intensity of deprivation (%)"
116,101006,Population in severe multidimensional poverty (%)
122,142506,Population vulnerable to multidimensional poverty (%)


In [6]:
# loading the Country codes
country_col_names = ['Country_Code', 'Country_Name']
country_UNDP_df = pd.read_html(my_data_catalog_path)[1][2:]
country_UNDP_df.reset_index(inplace = True, drop = True) # fix the indexing
country_UNDP_df.columns = country_col_names # Fix columns names
country_UNDP_df.head()

,Country_Code,Country_Name
0,AFG,Afghanistan
1,ALB,Albania
2,DZA,Algeria
3,AND,Andorra
4,AGO,Angola


In [7]:
country_UNDP_df.shape

(195, 2)

In [ ]:
pd.DataFrame(data=response.json(),
            columns = ['indicator_id', 
                       'indicator_name', 
                       'country_code',
                        'country_name',
                        'year',
                        'value'])

### Define a function that call a specific parameter and retunr a df

In [8]:
url_seed = "http://ec2-54-174-131-205.compute-1.amazonaws.com/API/HDRO_API.php/"
years = np.arange(2010, 2020)
years = ','.join(map(str, years))
indicator_id = "195606"
struct = ["ciy", "yic", "yci", "iyc", "icy"] 

In [75]:
#url_target = url_seed + "indicator_id" + "=" + indicator_id + "/" + "year" + "=" + years + "/" + "structure" + "=" + struct[2]

In [17]:
url_target = url_seed  + 'country_code=AFG,ZWE/' + "indicator_id" + "=" + indicator_id + "/" + "year" + "=" + years + "/" + "structure" + "=" + struct[3]
#url_target = "http://ec2-54-174-131-205.compute-1.amazonaws.com/API/HDRO_API.php/country_code=AFG,ZWE/indicator_id=103206,103606,103706/year=1990,2013,2019/structure=ciy"

In [18]:
# defining session and user Agent
s = requests.Session()
#s.auth = ('rjlopez2@gmail.com', 'EM@KU2%pM7GAccPlf!')
headers = {
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36"
    
}
response = requests.get(url_target, headers=headers)
if response.ok:
    print ('all OK!')
else:
    print ('Problem retrieving the website. Response status --->', response.status_code)
    

all OK!


In [19]:
json_data = response.json()

In [429]:
def API_UNDP_call_to_df(url_seed, indic_code, years = years):
    # define User agent and session
    s = requests.Session()
    headers = {
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36"
    }
    # create parameterized query to the API
    
    my_url = url_seed + "indicator_id" + "=" + indic_code +  "/" + "year" + "=" + years
    
    response = requests.get(my_url, headers=headers)
    
    my_df = pd.DataFrame(data=response.json(),
            columns = ['indicator_value', 
                       'country_name',
                        'indicator_name'])
#     print(response.json()[1])
    return(my_df)
    

In [430]:
df_1 = API_UNDP_call_to_df(url_seed = url_seed, 
                   indic_code = "38406")
#                    years = '2000')

In [35]:
 #np.arange(2010, 2020)
print(years)
list( map(str, np.arange(2010, 2020)))

2018,2019


['2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019']

In [12]:
pd.json_normalize(response.json(), meta = ['country_name','indicator_name', 'indicator_value'])

,indicator_value.2010.AFG.195606,indicator_value.2010.ZWE.195606,indicator_value.2011.AFG.195606,indicator_value.2011.ZWE.195606,indicator_value.2012.AFG.195606,indicator_value.2012.ZWE.195606,indicator_value.2013.AFG.195606,indicator_value.2013.ZWE.195606,indicator_value.2014.AFG.195606,indicator_value.2014.ZWE.195606,...,indicator_value.2015.ZWE.195606,indicator_value.2016.AFG.195606,indicator_value.2016.ZWE.195606,indicator_value.2017.AFG.195606,indicator_value.2017.ZWE.195606,indicator_value.2018.AFG.195606,indicator_value.2018.ZWE.195606,country_name.AFG,country_name.ZWE,indicator_name.195606
0,0.29,0.607,0.406,0.737,0.345,0.587,0.28,0.871,0.253,0.88,...,0.88,0.245,0.77,0.247,0.845,0.254,0.849,Afghanistan,Zimbabwe,"Carbon dioxide emissions, production emissions per capita (tonnes)"


In [ ]:
pd.json_normalize(response.json(), record_path = "195606")

In [443]:
pd.json_normalize(json_data['indicator_value'], record_path = ['195606'])

KeyError: '195606'

In [13]:
pd.DataFrame(json_data['indicator_value']).melt()

,variable,value
0,2010,{'195606': 0.29}
1,2010,{'195606': 0.607}
2,2011,{'195606': 0.406}
3,2011,{'195606': 0.737}
4,2012,{'195606': 0.345}
5,2012,{'195606': 0.587}
6,2013,{'195606': 0.28}
7,2013,{'195606': 0.871}
8,2014,{'195606': 0.253}
9,2014,{'195606': 0.88}


In [14]:
pd.json_normalize(json_data['indicator_value'], meta = ['country_name','indicator_name', 'indicator_value'])

,2010.AFG.195606,2010.ZWE.195606,2011.AFG.195606,2011.ZWE.195606,2012.AFG.195606,2012.ZWE.195606,2013.AFG.195606,2013.ZWE.195606,2014.AFG.195606,2014.ZWE.195606,2015.AFG.195606,2015.ZWE.195606,2016.AFG.195606,2016.ZWE.195606,2017.AFG.195606,2017.ZWE.195606,2018.AFG.195606,2018.ZWE.195606
0,0.29,0.607,0.406,0.737,0.345,0.587,0.28,0.871,0.253,0.88,0.262,0.88,0.245,0.77,0.247,0.845,0.254,0.849


In [15]:
json_data

{'indicator_value': {'2010': {'AFG': {'195606': 0.29},
   'ZWE': {'195606': 0.607}},
  '2011': {'AFG': {'195606': 0.406}, 'ZWE': {'195606': 0.737}},
  '2012': {'AFG': {'195606': 0.345}, 'ZWE': {'195606': 0.587}},
  '2013': {'AFG': {'195606': 0.28}, 'ZWE': {'195606': 0.871}},
  '2014': {'AFG': {'195606': 0.253}, 'ZWE': {'195606': 0.88}},
  '2015': {'AFG': {'195606': 0.262}, 'ZWE': {'195606': 0.88}},
  '2016': {'AFG': {'195606': 0.245}, 'ZWE': {'195606': 0.77}},
  '2017': {'AFG': {'195606': 0.247}, 'ZWE': {'195606': 0.845}},
  '2018': {'AFG': {'195606': 0.254}, 'ZWE': {'195606': 0.849}}},
 'country_name': {'AFG': 'Afghanistan', 'ZWE': 'Zimbabwe'},
 'indicator_name': {'195606': 'Carbon dioxide emissions, production emissions per capita (tonnes)'}}

In [21]:
json_data['indicator_value']['195606']

{'2010': {'AFG': 0.29, 'ZWE': 0.607},
 '2011': {'AFG': 0.406, 'ZWE': 0.737},
 '2012': {'AFG': 0.345, 'ZWE': 0.587},
 '2013': {'AFG': 0.28, 'ZWE': 0.871},
 '2014': {'AFG': 0.253, 'ZWE': 0.88},
 '2015': {'AFG': 0.262, 'ZWE': 0.88},
 '2016': {'AFG': 0.245, 'ZWE': 0.77},
 '2017': {'AFG': 0.247, 'ZWE': 0.845},
 '2018': {'AFG': 0.254, 'ZWE': 0.849}}

In [25]:
pd.json_normalize(json_data['indicator_value']['195606'], meta = ['country_name','indicator_name', 'indicator_value'])

,2010.AFG,2010.ZWE,2011.AFG,2011.ZWE,2012.AFG,2012.ZWE,2013.AFG,2013.ZWE,2014.AFG,2014.ZWE,2015.AFG,2015.ZWE,2016.AFG,2016.ZWE,2017.AFG,2017.ZWE,2018.AFG,2018.ZWE
0,0.29,0.607,0.406,0.737,0.345,0.587,0.28,0.871,0.253,0.88,0.262,0.88,0.245,0.77,0.247,0.845,0.254,0.849


In [85]:
pd.json_normalize(json_data['indicator_value'], meta = ['country_name', 'indicator_name'])

,2010.AFG.195606,2010.ZWE.195606,2011.AFG.195606,2011.ZWE.195606,2012.AFG.195606,2012.ZWE.195606,2013.AFG.195606,2013.ZWE.195606,2014.AFG.195606,2014.ZWE.195606,2015.AFG.195606,2015.ZWE.195606,2016.AFG.195606,2016.ZWE.195606,2017.AFG.195606,2017.ZWE.195606,2018.AFG.195606,2018.ZWE.195606
0,0.29,0.607,0.406,0.737,0.345,0.587,0.28,0.871,0.253,0.88,0.262,0.88,0.245,0.77,0.247,0.845,0.254,0.849


In [80]:
pd.json_normalize(json_data, '195606')

KeyError: '195606'

In [72]:
pd.DataFrame(json_data).melt()

,variable,value
0,indicator_value,"{'AFG': {'2010': 0.29, '2011': 0.406, '2012': 0.345, '2013': 0.28, '2014': 0.253, '2015': 0.262, '2016': 0.245, '2017': 0.247, '2018': 0.254}, 'ZWE': {'2010': 0.607, '2011': 0.737, '2012': 0.587, '2013': 0.871, '2014': 0.88, '2015': 0.88, '2016': 0.77, '2017': 0.845, '2018': 0.849}}"
1,indicator_value,NaN
2,indicator_value,NaN
3,country_name,NaN
4,country_name,Afghanistan
5,country_name,Zimbabwe
6,indicator_name,"Carbon dioxide emissions, production emissions per capita (tonnes)"
7,indicator_name,NaN
8,indicator_name,NaN


In [30]:
os.path.abspath('.')

'/Users/rubencito/CAS_datascience/ADS_CAS_Bern_2020/Projects/M3'

In [32]:
OECD_df = pd.read_csv(os.path.abspath('.') + "/Vale_data/CAS_2021/GDP_OECD_countries.csv")

In [33]:
OECD_df.shape

(375540, 17)

In [34]:
OECD_df.head(10)

,LOCATION,Country,VARIABLE,Variable,FREQUENCY,Frequency,TIME,Time,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,BSII,Balance of primary income (balance of payments basis),A,Annual,1960,1960,AUD,Australian Dollar,0,Units,NaN,NaN,-248000000.0,NaN,NaN
1,AUS,Australia,BSII,Balance of primary income (balance of payments basis),A,Annual,1961,1961,AUD,Australian Dollar,0,Units,NaN,NaN,-250000000.0,NaN,NaN
2,AUS,Australia,BSII,Balance of primary income (balance of payments basis),A,Annual,1962,1962,AUD,Australian Dollar,0,Units,NaN,NaN,-243000000.0,NaN,NaN
3,AUS,Australia,BSII,Balance of primary income (balance of payments basis),A,Annual,1963,1963,AUD,Australian Dollar,0,Units,NaN,NaN,-296000000.0,NaN,NaN
4,AUS,Australia,BSII,Balance of primary income (balance of payments basis),A,Annual,1964,1964,AUD,Australian Dollar,0,Units,NaN,NaN,-279000000.0,NaN,NaN
5,AUS,Australia,BSII,Balance of primary income (balance of payments basis),A,Annual,1965,1965,AUD,Australian Dollar,0,Units,NaN,NaN,-310000000.0,NaN,NaN
6,AUS,Australia,BSII,Balance of primary income (balance of payments basis),A,Annual,1966,1966,AUD,Australian Dollar,0,Units,NaN,NaN,-303000000.0,NaN,NaN
7,AUS,Australia,BSII,Balance of primary income (balance of payments basis),A,Annual,1967,1967,AUD,Australian Dollar,0,Units,NaN,NaN,-405000000.0,NaN,NaN
8,AUS,Australia,BSII,Balance of primary income (balance of payments basis),A,Annual,1968,1968,AUD,Australian Dollar,0,Units,NaN,NaN,-495000000.0,NaN,NaN
9,AUS,Australia,BSII,Balance of primary income (balance of payments basis),A,Annual,1969,1969,AUD,Australian Dollar,0,Units,NaN,NaN,-524000000.0,NaN,NaN
